Reading classics [Python Machine Learning 3rd Edition](https://github.com/rasbt/python-machine-learning-book-3rd-edition/blob/master/ch14/ch14_part2.ipynb)

In [ ]:
%%writefile cmap_header.py
from IPython.display import display,HTML
from IPython.core.magic import register_line_magic
import random
#Sequential (Single-Hue,Multi-Hue),Diverging, Cyclical cmaps: 
#Reds,Sinebow,Rainbow,Turbo,Warm,Cool,Plasma,Spectral,etc.
@register_line_magic
def cmap_header(params):
    params=params.split('|'); string=params[0]
    if len(params)==1: 
        font_size='30'; font_family='Akronim'; cmap='Sinebow'
    elif  len(params)==2: 
        font_size=params[1]
        font_family='Akronim'; cmap='Sinebow'
    elif  len(params)==3: 
        font_size=params[1]; font_family=params[2]
        cmap='Sinebow'
    else: 
        font_size=params[1]; font_family=params[2]; cmap=params[3]
    height=int(font_size)*2.5; randi=str(random.uniform(0,9999999))
    html_str="""
    <head><script src='https://d3js.org/d3.v6.min.js'></script>
    </head><style>@import 'https://fonts.googleapis.com/css?family="""+\
    font_family+"""&effect=3d'; #colorized {font-family:"""+font_family+\
    """; color:white; padding-left:10px; font-size:"""+font_size+\
    """px;}</style><h1 id='colorized' class='font-effect-3d'>"""+\
    string+"""</h1><script>
    var tc=setInterval(function(){
        var now=new Date().getTime();
        var iddoc=document.getElementById('colorized');
        iddoc.style.color=d3.interpolate"""+cmap+\
    """(now%(30000)/30000);},1)</script>"""
    file='d3header'+randi+'.html'
    with open(file,'w') as f:
         f.write(html_str); f.close()
    string="""<div id='html_string"""+randi+\
    """' style='width:100%;'><iframe src='"""+file+\
    """' height="""+str(height)+"""
    style='display:block; width:100%;'></iframe></div>"""
    display(HTML(string))

In [ ]:
%run cmap_header.py
%cmap_header Code Modules & Functions

In [ ]:
import tensorflow as tf,pylab as pl
import pandas as pd,numpy as np,seaborn as sb
import tensorflow.keras.utils as tku
from tensorflow.data import Dataset as tds
import sklearn.model_selection as sms

In [ ]:
def phtml(str):
    display(HTML("""<style>@import 
    'https://fonts.googleapis.com/css?family=Aladin&effect=3d-float';      
    </style><h1 class='font-effect-3d-float' 
    style='font-family:Aladin; color:#ee55ee; font-size:20px;'>
    %s</h1>"""%str))
def pd_style():
    return [dict(selector='th',props=[('font-size','10pt')]),
            dict(selector='td',props=[('padding','0em 0em')]),
            dict(selector='th:hover',props=[('font-size','12pt')]),
            dict(selector='tr:hover td:hover',
                 props=[('max-width','200px'),('font-size','10pt')])]

In [ ]:
%cmap_header Data Processing

In [ ]:
url='http://archive.ics.uci.edu/ml/machine-learning-databases'+\
    '/auto-mpg/auto-mpg.data'
dp=tku.get_file('auto-mpg.data',url)

In [ ]:
columns=['MPG','Cylinders','Displacement','Horsepower',
         'Weight','Acceleration','ModelYear','Origin']
numeric_columns=columns[2:6]
df=pd.read_csv(dp,names=columns,
               na_values="?",comment='\t',sep=" ",
               skipinitialspace=True)
df.head(10).style.background_gradient('cool',axis=1)\
    .set_properties(**{'max-width':'80px','font-size':'1pt'})\
    .set_caption(phtml('Data Exploration'))\
    .set_precision(2).set_table_styles(pd_style())

In [ ]:
df=df.dropna().reset_index(drop=True)
df.tail(10).style.background_gradient(
    cmap='cool',axis=0,subset=numeric_columns)\
.set_table_styles(pd_style())

In [ ]:
stats=df.describe().transpose()
dfn=df.copy()
for nc in numeric_columns:
    mean=stats.loc[nc,'mean']
    std=stats.loc[nc,'std']
    dfn.loc[:,nc]=(dfn.loc[:,nc]-mean)/std
dfn.tail(10).style.bar(
    align='mid',color=['#55eeee','#ee55ee'],
    subset=numeric_columns).set_table_styles(pd_style())

In [ ]:
%cmap_header TF Data

In [ ]:
numeric_features=[]
for nc in numeric_columns:
    nf=tf.feature_column.numeric_column(key=nc)
    numeric_features.append(nf) 
for nf in numeric_features: 
    phtml(str(nf)); break

In [ ]:
fyear=tf.feature_column.numeric_column(key="ModelYear")
fcylinders=tf.feature_column.numeric_column(key="Cylinders")
bucketized_features=[]
fyear=tf.feature_column.bucketized_column(
    source_column=fyear,boundaries=[73,76,79])
fcylinders=tf.feature_column.bucketized_column(
    source_column=fcylinders,boundaries=[4,6,8])
bucketized_features.append(fyear)
bucketized_features.append(fcylinders)
for bf in bucketized_features: 
    phtml(str(bf)); break

In [ ]:
forigin=tf.feature_column\
.categorical_column_with_vocabulary_list(
    key='Origin',vocabulary_list=[1,2,3])
categorical_features=\
[tf.feature_column.indicator_column(forigin)]
phtml(str(categorical_features[0]))

In [ ]:
features=(numeric_features+bucketized_features+categorical_features)
dfn_train,dfn_test=\
sms.train_test_split(dfn,train_size=.8,shuffle=True)
phtml(str([len(dfn_train),len(dfn_test)]))

In [ ]:
batch_size=8
def train_input_fn(dfn_train,batch_size):
    dft=dfn_train.copy()
    x_train,y_train=dft,dft.pop('MPG')
    ds_train=tds.from_tensor_slices((dict(x_train),y_train))
    return ds_train.shuffle(len(x_train)).repeat().batch(batch_size)
ds_train=train_input_fn(dfn_train,batch_size)
batch=next(iter(ds_train))
phtml('keys: </br>%s'%batch[0].keys())
phtml('batch values of `Cylinders`: </br> %s'%batch[0]['Cylinders'])

In [ ]:
def test_input_fn(dfn_test,batch_size):
    dft2=dfn_test.copy()
    x_test,y_test=dft2,dft2.pop('MPG')
    ds_test=tds.from_tensor_slices((dict(x_test),y_test))
    return ds_test.batch(batch_size)
ds_test=test_input_fn(dfn_test,batch_size)
batch=next(iter(ds_test))
phtml('keys: </br>%s'%batch[0].keys())
phtml('batch values of `ModelYear`: </br>%s'%\
      batch[0]['ModelYear'])

In [ ]:
%cmap_header TF Regressor

In [ ]:
reg=tf.estimator.DNNRegressor(
    feature_columns=features,hidden_units=[64,32,16,8],
    model_dir='models/autompg-dnnregressor/');

In [ ]:
epochs=1000
steps=epochs*int(np.ceil(len(dfn_train)/batch_size))
phtml('training steps: %d'%steps)
reg.train(input_fn=lambda:\
          train_input_fn(dfn_train,
          batch_size),steps=steps);

In [ ]:
reloaded_reg=tf.estimator.DNNRegressor(
    feature_columns=features,hidden_units=[64,32,16,8],
    warm_start_from='models/autompg-dnnregressor/',
    model_dir='models/autompg-dnnregressor/');

In [ ]:
test_results=reloaded_reg\
.evaluate(input_fn=lambda:\
          test_input_fn(dfn_test,batch_size))
for key in test_results:
    phtml('{}: {}'.format(key,test_results[key]))
phtml('test average loss% {:.4f}'\
      .format(test_results['average_loss']))

In [ ]:
y_test_pred=reg.predict(input_fn=lambda:\
                        test_input_fn(dfn_test,batch_size))
py_test=[]
for i in range(len(dfn_test)):
    py_test.append(next(iter(y_test_pred))['predictions'][0])

In [ ]:
pl.figure(figsize=(10,4))
pl.plot(range(len(dfn_test)),dfn_test['MPG'],
        '-o',label='real data',c='#55eeee',ms=7)
pl.plot(range(len(dfn_test)),py_test,
        '-o',label='predictions',c='#ee55ee',ms=5)
pl.tight_layout(); pl.grid(); pl.legend();